<a href="https://colab.research.google.com/github/rsadaphule/jhu-dnn/blob/main/%5BGroup_15_Final_Submission%5D_8%F0%9F%8F%86%E2%98%A2%EF%B8%8FRadon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>☢️Radon</font>](https://www.kaggle.com/c/17jul23jh-radon/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle >> log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >>log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 17jul23jh-radon # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 17jul23jh-radon
Using competition: 17jul23jh-radon
  teamId  teamName              submissionDate       score      
--------  --------------------  -------------------  ---------  
10718785  Group 15              2023-07-30 00:34:06  23.05968   
10720520  Group 8               2023-07-30 02:50:29  24.30620   
10723884  Group 4               2023-07-30 12:46:17  27.27605   
10728215  Jin C                 2023-07-30 05:31:11  27.32239   
10717925  Sanddhya Jayabalan    2023-07-30 17:12:43  27.98797   
10719289  JaneC                 2023-07-29 17:21:43  28.80044   
10724015  erykbanatt            2023-07-28 22:47:08  28.83907   
10718607  Matt Sunday           2023-07-30 02:50:40  30.39904   
10727378  Group3                2023-07-30 16:46:55  33.75676   
10717755  Group 6               2023-07-30 12:46:55  35.21138   
10719741  Vincent Kowalski      2023-07-28 23:30:25  35.23181   
10735853  Lu Liu  

In [ ]:
# !pip list keras|grep keras

In [ ]:
# !pip install keras==2.12.0 >> log
# !pip list keras|grep keras

In [ ]:
!pip install -U tfds-nightly tensorflow_addons tensorflow keras==2.12.0 >> log # downgrade keras to bypass tf_utils import error

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, tensorflow_addons as tfa, tensorflow as tf, tensorflow.keras as keras, os
from keras.layers import Flatten, Dense
os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'; # allows seeding RNG on GPU
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 3.1 s, sys: 449 ms, total: 3.55 s
Wall time: 3.61 s


In [ ]:
df_raw = pd.read_csv('XY_radon.csv'); df_raw

,Uppm,adjwt,basement,cntyfips,county,dupflag,floor,lat,lon,pcterr,...,stfips,stopdt,stoptm,stratum,typebldg,wave,windoor,zip,zipflag,Y
0,1.80,54.97,Y,59,MORTON,0,1,46.66,-101.39,7.76,...,38,32206,1230,2,2,1,NaN,58554,0,NaN
1,1.65,499.34,N,85,KOSCIUSKO,0,1,40.85,-86.22,55.02,...,18,11497,1430,3,1,32,NaN,46580,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12571,0.44,394.07,Y,3,ANOKA,0,0,44.91,-92.86,9.40,...,27,32110,1500,2,1,4,NaN,55303,0,8.6
12572,2.71,157.82,Y,15,MOHAVE,0,0,36.01,-113.21,14.46,...,4,11496,1330,1,1,38,NaN,86403,0,1.9


In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Approach**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**NOTE: Notebook should be run using GPU**

**1. Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.**

Our preprocessing consists of outlier removal and feature engineering.

**-Outlier removal:** First, we noticed that the pcterr value has larger values present in the training set but not the test set and decided to remove outliers by clipping the upper bound.

**-Feature engineering:** We took the log of the pcterr feature since the original values exhibited a heavy positive skew. We also created new features such as lat_high, lon_high, basement_zero, stratum_one and region_one which create a binary variable based on the value of the respective feature.

Given that the nature of the problem is highly dependent on the geographical location, we computed a global FIPS value based on the state and zip fips. In essence, we combined the two features together to contain both statewide and countywide information.

Finally, we computed mean values (column-wise) for each county and cntyfips. For example, county_mean_uranium and cntyfips_mean_uranium are examples of how we used column wise mean values grouped on county and cntyfips.

**2. Why did you choose these elements? (Perhaps something in EDA or prior experience lead you to these)**

We noticed that the pcterr values have a large positive skew so taking the log of the pcterr makes it closer to a gaussian distribution, which is easier for the neural network to learn from.

The feature that led to a sizable improvement in performance was a binary feature that represented log_pcterr = 0. We analyzed the mistakes being made by an earlier model, and in doing so, we noticed that log_pcterr and y were negatively correlated: lower log_pcterr had higher y values, with the exception of log_pcterr = 0, which had low y values. Once we added in this feature, we saw a noticeable jump in both CV and Kaggle LB scores.

Also, we chose features based on the nature of the dataset. The dataset we are analyzing is heavily dependent on geographical location. Therefore, we calculated mean values based on the geographical locations. This brings in information about the surrounding area as well as the specific instance we are trying to analyze.

**3. How do you evaluate the effectiveness of these elements?**

We evaluated the effectiveness of these elements by trying various configurations and running 5-fold cross validation in order to compute the mean squared error, and logging the performance in our notebook. It was an iterative approach to find novel ways to engineer the new features. A lot of effort went into feature engineering because we noticed that the model performance did not vary drastically with different configurations of model depth or hyperparameters (see details in next section).

**4 What else have you tried that worked or didn't?**

We tried to keep the data as it is in the baseline model and fine tune the model configuration; however, we noticed that the original baseline model was already configured well. Therefore there was no real value gain from just updating our model. We had to take a look at the features and find novel ways to engineer our data.

The Kaggle dataset offers some insights into the data and the problem at hand. The documentation mentioned that the way radon enters houses is through its contact at various points such as the basement and floor as two examples. Given this information we decided to one hot encode the floor - as opposed to the sequential ordering it originally.

We also tried clustering the data based on geographical location such as the latitude, longitude, cntyfips and statefips. The idea was that we could cluster the database on its geographical location. Given the importance of geographical location for this dataset the goal was to capture more granular geographical location however it had negligible effect.

We also followed the suggestion to include distance to Radon source as a feature [2]. We found this was only a useful feature after grouping by state; but ultimately, there was not a strong enough correlation or improvement in the accuracy score to include this feature. This weak correlation can be seen in the below heatmaps.


![](https://drive.google.com/uc?export=view&id=1W3E3dtKftxMni8rulVlrmf0HWeKw8vz-)

**Student's answer:**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from collections import Counter
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras import layers
from keras.layers import BatchNormalization, Dropout, Activation
import numpy as np, pandas as pd, tensorflow as tf, plotly.express as px, platform, plotly, os, time
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from joblib import parallel_backend

# pd.set_option('display.max_rows', None)

In [ ]:
%%time
import os
tf.random.set_seed(42) # always seed your experiments
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'; # allows seeding RNG on GPU

CPU times: user 175 µs, sys: 27 µs, total: 202 µs
Wall time: 210 µs


### RUN: Create data

In [ ]:
df = df_raw.drop(['windoor','zipflag'], axis=1)  # drop non-numeric columns
vX = df.query('Y!=Y').drop('Y', axis=1)    # slice a test sample
tXY = df.query('Y==Y')                     # slice training sample
tX, tY = tXY.drop('Y', axis=1), tXY.Y      # split into training I/O

# tXY.head()                  # train outputs

### RUN: Feature eng & Outlier removal

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn

# Outlier removal
tXY = tXY[tXY.pcterr<150]

# Feature engineering: add new features
tXY['log_pcterr'] = np.log(tXY['pcterr']+1)
vX['log_pcterr'] = np.log(vX['pcterr']+1)

tXY['basement_num'] = np.where(tXY.basement=='Y',1,0)
vX['basement_num'] = np.where(vX.basement=='Y',1,0)

tXY['fips'] = tXY['stfips'] * 1000 + tXY['cntyfips']
vX['fips'] = vX['stfips'] * 1000 + vX['cntyfips']

tXY['zero_log_pcterr'] = (tXY['log_pcterr']==0).astype('int')
vX['zero_log_pcterr'] = (vX['log_pcterr']==0).astype('int')

tXY['high_adjwt'] = (tXY['adjwt']>1700).astype('int')
vX['high_adjwt'] = (vX['adjwt']>1700).astype('int')

tXY['lon_low'] = (tXY['lon']<-105).astype('int')
vX['lon_low'] = (vX['lon']<-105).astype('int')
tXY['lon_high'] = (tXY['lon']>-82).astype('int')
vX['lon_high'] = (vX['lon']>-82).astype('int')

tXY['lat_high'] = (tXY['lat']>42.5).astype('int')
vX['lat_high'] = (vX['lat']>42.5).astype('int')

tXY['wave_low'] = (tXY['wave']==-1).astype('int')
vX['wave_low'] = (vX['wave']==-1).astype('int')

tXY['floor_zero'] = (tXY['floor']==0).astype('int')
vX['floor_zero'] = (vX['floor']==0).astype('int')

tXY['basement_zero'] = (tXY['basement_num']==0).astype('int')
vX['basement_zero'] = (vX['basement_num']==0).astype('int')

tXY['stratum_one'] = (tXY['stratum']==1).astype('int')
vX['stratum_one'] = (vX['stratum']==1).astype('int')

tXY['region_one'] = (tXY['region']==1).astype('int')
vX['region_one'] = (vX['region']==1).astype('int')

# tXY.shape

### RUN: Derived features calculations

In [ ]:
# Compute new features by aggregating/ averaging at the county or cntyfips level
def computations(df, col):
  # Calculate % homes with basement at col level
  has_basement = pd.DataFrame(df[col].value_counts()).reset_index()
  has_basement.columns = [col, 'county_count']
  temp_has_basement = pd.DataFrame(df[df.basement_num>0][col].value_counts()).reset_index()
  temp_has_basement.columns = [col, 'has_base']

  has_basement = has_basement.merge(temp_has_basement, on=col, how='left')
  has_basement[col+'_pct_has_basement'] = has_basement['has_base'] / has_basement['county_count']
  has_basement = has_basement.drop(['county_count', 'has_base'], axis=1).fillna(0)

  # Calculate % homes with >0 floor at col level
  has_floor = pd.DataFrame(df[col].value_counts()).reset_index()
  has_floor.columns = [col, 'county_count']
  temp_has_floor = pd.DataFrame(df[df.floor>0][col].value_counts()).reset_index()
  temp_has_floor.columns = [col, 'has_floor']

  has_floor = has_floor.merge(temp_has_floor, on=col, how='left')
  has_floor[col+'_pct_has_floor'] = has_floor['has_floor'] / has_floor['county_count']
  has_floor = has_floor.drop(['county_count', 'has_floor'], axis=1).fillna(0)

  # Calculate avg uranum at col level
  mean_uranium = df[[col,'Uppm']].groupby(col).mean().reset_index().fillna(0)
  mean_uranium.columns = [col,col+'_mean_uranium']

  # Calculate avg adjwt at col level
  mean_adjwt = df[[col,'adjwt']].groupby(col).mean().reset_index().fillna(0)
  mean_adjwt.columns = [col,col+'_mean_adjwt']

  # NEW: Calculate avg pcterr at col level
  mean_pcterr = df[[col,'pcterr']].groupby(col).mean().reset_index().fillna(0)
  mean_pcterr.columns = [col,col+'_pcterr']
  df = df.merge(mean_pcterr, on=col, how='left')

  df = df.merge(has_basement, on=col, how='left')
  df = df.merge(has_floor, on=col, how='left')
  df = df.merge(mean_uranium, on=col, how='left')
  df = df.merge(mean_adjwt, on=col, how='left')

  return df

In [ ]:
tXY = computations(tXY, 'cntyfips')
vX = computations(vX, 'cntyfips')
tXY = computations(tXY, 'county')
vX = computations(vX, 'county')
# tXY.head()

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**1. Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.**

We decided to use a neural network and after some experimentation (detailed below) observed that a small model would work best for this problem. This aligned with our expectations about how neural networks perform: given that there are only 6285 training observations and less than 50 features, a deep model would not work well for this problem. Therefore we started exploring networks with fewer parameters and settled on a model with only one hidden layer. This simple model combined with feature engineering performed significantly better than a more complex model without any feature engineering.

We performed hyperparameter tuning on our model, testing different values for the number of neurons in the hidden layer, the dropout, batch size and learning rate. We found that the leaky relu activation function performed best and also utilized an exponential learning rate decay.
We analyzed initializations to ensure they were configured correctly. For example, it was preferable to use He_Normal initialization when using a Relu or variation of Relu (leaky-relu). We were able to come to this conclusion based on our previous knowledge of neural networks and how they work.

In the prediction outputs, we clip the lower bound of the predictions at 0; otherwise, the model outputs small negative values of y.

**2. How did these decisions guide you in modeling?**

The decisions mentioned above were largely based on our intuition on how neural networks performed with different datasets. As to which decisions we ended up going with, model performance and runtime were two main factors. For example, we knew based on the size of the data that a simpler neural network with fewer nodes than a deep neural network would perform just as well but also run much faster. This led us to experimenting with different hyperparameters of a smaller neural network as described above. In all, we were able to test a variety of hypotheses and see which performed best.

**3. How do you evaluate the effectiveness of these elements?**

We evaluated the effectiveness of these elements using 5 fold cross validation to assess how well the model performed on average as measured by the mean squared error. We plotted this in a learning plot over the epochs. We also plotted the mean absolute error as a guardrail metric. We recorded the performance of different techniques and approaches to see how various techniques ranked against one another.

We also made submissions on Kaggle to see how our cross validation scores compared to the public leaderboard scores.

**4. What else have you tried that worked or didn't?**

The baseline model consisted of three hidden layers with 5 neurons per layer. At first we thought that a more complex model could perform better, so added more layers, more neurons, batch normalization layers and dropout layers. We tried hyperparameter tuning the model using random search over our parameter grid. Some of the parameters we tuned are batch size, epochs, learning rate, optimizers, dropout %, and number of neurons. However this did not provide significant lift in performance vs. the simple baseline model. From this, we realized that a more complex model does not necessarily perform better.  

We observed that some of the predicted values were less than 0 so we had the idea of using a custom sigmoid activation function in the output layer to constrain the predictions, stretching the upper bound to 280 since this is the highest y value in the training data. However, we observed that while this did manage to cap the lowest values at zero, there were high errors for large y values. So we abandoned this approach in favor of our final approach.

![](https://drive.google.com/uc?export=view&id=1T7fFcQjDHNkTLhNmwxJHvGegkXuX5Z_8)

**Student's answer:**

### RUN: Define training data

In [ ]:
## DEFINE TRAINING DATA
include_cols = [
    # Location
    'fips',
    # House
    'floor',  'room', 'basement_num',
    # Measures
    'Uppm',  'log_pcterr',
    # Other
    'region', 'rep',
    'stratum', 'typebldg', 'wave',
    # Derived
    'county_mean_uranium',
    'cntyfips_mean_uranium',
    'zero_log_pcterr', 'high_adjwt', 'lon_low', 'lon_high','lat_high', 'wave_low',
    'floor_zero','basement_zero','stratum_one','region_one',
    # NEW
    'cntyfips_pcterr',
    'county_pcterr',
    ]
input_X = tXY[include_cols]
input_X.county_mean_uranium = input_X.county_mean_uranium.fillna(tXY.Uppm)
input_X.cntyfips_mean_uranium = input_X.cntyfips_mean_uranium.fillna(tXY.Uppm)

test_X = vX[include_cols]
test_X.county_mean_uranium = test_X.county_mean_uranium.fillna(tXY.Uppm)
test_X.cntyfips_mean_uranium = test_X.cntyfips_mean_uranium.fillna(tXY.Uppm)

# NEW
input_X.cntyfips_pcterr = input_X.cntyfips_pcterr.fillna(tXY.pcterr)
input_X.county_pcterr = input_X.county_pcterr.fillna(tXY.pcterr)
test_X.cntyfips_pcterr = test_X.cntyfips_pcterr.fillna(tXY.pcterr)
test_X.county_pcterr = test_X.county_pcterr.fillna(tXY.pcterr)

input_Y = tXY.Y

# input_X.columns

### RUN: Define NN model params

In [ ]:
LEARNING_RATE = 0.05
NEURONS = 256
BATCH_SIZE = 128
DROPOUT = 0.4
EPOCH = 30

# Set a exponential decay for the learning rate
def expDecayLR(eta0=LEARNING_RATE, s=10):
  return lambda epoch: eta0 * 0.1**(epoch / s)  # returns a functional

# # Define custom sigmoid activation function. 280 is max y value. 290
# def sigmoid_stretched(x):
#   return (1.0/(1 + tf.math.exp(-x))) * 290

### Kfold CV

In [ ]:
# # Kfold CV
# %%time
# num_folds = 5

# X = input_X.reset_index(drop=True)
# Y = input_Y.reset_index(drop=True)

# # Stratified kfold
# # kf = StratifiedKFold(n_splits=num_folds, shuffle=True)
# # kf.get_n_splits(X, Y)

# # Regular kfold
# kf = KFold(n_splits=5, shuffle=True)
# kf.get_n_splits(X)

# # K-fold Cross Validation model evaluation
# fold_no = 1
# # Define per-fold score containers
# split_performance = []
# split_performance2 = []

# # Loop over the dataset to create seprate folds
# for train_index, test_index in kf.split(X): #, Y): # Need this for stratified kfold
#   # print(test_index)
#   X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#   y_train, y_test = [Y[i] for i in train_index], [Y[i] for i in test_index]

#   train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
#   validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
#   train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=False)
#   validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=False)

#   tf.random.set_seed(0)   # always seed your experiments
#   Init = keras.initializers.RandomNormal(seed=0)
#   m = keras.models.Sequential([
#       Flatten(input_shape=[input_X.shape[1]], name='input'),
#       # input_X_norm,
#       BatchNormalization(),
#       Dense(NEURONS, activation="relu", kernel_initializer="he_normal", name='hidden1'),
#       Dropout(DROPOUT),
#       BatchNormalization(),
#       Dense(1, activation="LeakyReLU", kernel_initializer=Init, name='output')])

#   compile_model = m.compile(
#       loss="mse",
#       optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm=1.),
#       metrics=['mean_absolute_error'])

#   # Generate a print
#   print('------------------------------------------------------------------------')
#   print(f'Training for fold {fold_no} ...')

#   hist = m.fit(
#       train_dataset,
#       validation_data = validation_dataset,
#       verbose = False,
#       epochs = EPOCH,
#       callbacks=[keras.callbacks.LearningRateScheduler(expDecayLR())])

#   print(hist.history['val_mean_absolute_error'][-1])
#   print(hist.history['val_loss'][-1])
#   split_performance.append(hist.history['val_mean_absolute_error'][-1])
#   split_performance2.append(hist.history['val_loss'][-1])

#   # Increase fold number
#   fold_no = fold_no + 1

# print('------------------------------------------------------------------------')
# print('Mean absolute error:', np.mean(split_performance), ', Var MAE =', np.var(split_performance))
# print('Mean validation loss:', np.mean(split_performance2), ', Var loss =', np.var(split_performance2))

### Run model & plot

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(input_X, input_Y, test_size=0.2, random_state=42)

In [ ]:
# %%time

# X_train
# # Build NN model with hidden layers, using Relu activation, dropout and Batch norm
# from joblib import parallel_backend

# tf.random.set_seed(0)   # always seed your experiments
# Init = keras.initializers.RandomNormal(seed=0)

# with parallel_backend('threading'):
#   m = keras.models.Sequential([
#       Flatten(input_shape=[input_X.shape[1]], name='input'),
#       BatchNormalization(),
#       Dense(NEURONS, activation="relu", kernel_initializer="he_normal", name='hidden1'),
#       Dropout(DROPOUT),
#       BatchNormalization(),
#       Dense(1, activation="LeakyReLU", kernel_initializer=Init, name='output')])

#   m.summary()
#   compile_model = m.compile(
#       loss="mse",
#       optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm=1.),
#       metrics=['mean_absolute_error'])
#   hist = m.fit(
#       X_train,
#       y_train,
#       epochs = EPOCH,
#       batch_size = BATCH_SIZE,
#       validation_data = (X_test,y_test),
#       # validation_split = 0.2,
#       callbacks=[keras.callbacks.LearningRateScheduler(expDecayLR())])

In [ ]:
# DConfig = {'modeBarButtonsToAdd':['drawline','drawopenpath','drawclosedpath','drawcircle','drawrect','eraseshape']} # more tool buttons for plotly

# print(f'Hyperparameters: ', hist.params)
# print(f'Epochs:          ', hist.epoch)
# print(f'History:         ', hist.history)
# dfHist = pd.DataFrame(hist.history, index=hist.epoch)  # create a datafrom from history dictionary
# f = px.scatter(dfHist[['mean_absolute_error', 'val_mean_absolute_error']][2:])
# f = f.update_traces(mode='lines+markers')
# f = f.update_layout(width=800, height=400, margin=dict(l=0, r=0, t=30, b=0), title='Learning performance')
# f.show(config=DConfig)

# f2 = px.scatter(dfHist[['loss', 'val_loss']][2:])
# f2 = f2.update_traces(mode='lines+markers')
# f2 = f2.update_layout(width=800, height=400, margin=dict(l=0, r=0, t=30, b=0), title='Learning performance')
# f2.show(config=DConfig)

### RUN: Train for submission

In [ ]:
%%time

# Build NN model with hidden layers, using Relu activation, dropout and Batch norm
from joblib import parallel_backend

tf.random.set_seed(0)   # always seed your experiments
Init = keras.initializers.RandomNormal(seed=0)

with parallel_backend('threading'):
  m = keras.models.Sequential([
      Flatten(input_shape=[input_X.shape[1]], name='input'),
      BatchNormalization(),
      Dense(NEURONS, activation="relu", kernel_initializer="he_normal", name='hidden1'),
      Dropout(DROPOUT),
      BatchNormalization(),
      Dense(1, activation="LeakyReLU", kernel_initializer=Init, name='output')])

  m.summary()
  compile_model = m.compile(
      loss="mse",
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm=1.),
      metrics=['mean_absolute_error'])
  hist = m.fit(
      input_X,
      input_Y,
      epochs = EPOCH,
      batch_size = BATCH_SIZE,
      callbacks=[keras.callbacks.LearningRateScheduler(expDecayLR())])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 25)                0         
                                                                 
 batch_normalization (BatchN  (None, 25)               100       
 ormalization)                                                   
                                                                 
 hidden1 (Dense)             (None, 256)               6656      
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 output (Dense)              (None, 1)                 2

In [ ]:
pred = m.predict(test_X)
pred = pd.DataFrame(pred).clip(lower=0)

197/197 [==============================] - 2s 6ms/step


In [ ]:
pY = pd.DataFrame(np.array(pred), index=np.arange(len(test_X))+1, columns=['y'])
ToCSV(pY, 'kewei_radon_july_30_validation')

In [ ]:
# from google.colab import files

# files.download("kewei_radon_july_30_validation.csv")

In [ ]:
# pY.describe().T

In [ ]:
# pd.DataFrame(tXY.Y.describe()).T

In [ ]:
# df = df_raw.select_dtypes(include=np.number).drop('windoor', axis=1)  # drop non-numeric columns
# vX = df.query('Y!=Y').drop('Y', axis=1)    # slice a test sample
# tXY = df.query('Y==Y')                     # slice training sample
# tX, tY = tXY.drop('Y', axis=1), tXY.Y.astype(int)      # split into training I/O
# print(tY.tolist()[:50])                    # train outputs

In [ ]:
# tf.random.set_seed(0)   # always seed your experiments
# Init = keras.initializers.RandomNormal(seed=0)

# m = keras.models.Sequential([
#   Flatten(input_shape=[tX.shape[1]]),
#   Dense(5, activation="relu", kernel_initializer=Init),
#   Dense(5, activation="relu", kernel_initializer=Init),
#   Dense(5, activation="relu", kernel_initializer=Init),
#   Dense(1, kernel_initializer=Init)])
# m.summary()
# m.compile(loss="mse", optimizer="adam", metrics=['mse'])
# # m.compile(loss=tf.keras.losses.Huber(), optimizer="adam", metrics=['mse'])

# hist = m.fit(
#     x=tX, y=tY,
#     batch_size=32,
#     epochs=5,
#     validation_split=0.3)

The model generates a baseline submission CSV file, see Colab folder (🗀 on the left), which you candownload and submit to Kaggle.

In [ ]:
# pY = pd.DataFrame(m.predict(vX), index=np.arange(len(vX))+1, columns=['y'])
# ToCSV(pY.round(0).astype(int), 'Radon_Baseline')

# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).

[1] Géron, A. (2019). Hands-on machine learning with Scikit-learn, Keras, and Tensorflow: Concepts, tools, and techniques to build intelligent systems (2nd edition). O'Reilly Media.

[2] Stack Overflow. (2011). Haversine Formula in Python (Bearing and Distance between two GPS points). Retrieved July 21, 2023, from https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points

[3] Tensorflow. Multilevel Modeling Primer in TensorFlow Probability, https://www.tensorflow.org/probability/examples/Multilevel_Modeling_Primer

[4] Stack Overflow. Training loss higher than validation loss. https://stackoverflow.com/questions/50387749/training-loss-higher-than-validation-loss

[5] Stack Overflow. Restricting the output values of layers in Keras. https://stackoverflow.com/questions/44553722/restricting-the-output-values-of-layers-in-keras

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 28 sec


# **Starter Ideas**

1. Try different DNN architectures and tuning of hyperparameters
1. Try converting locations to distances to the key Radon sources (which you might need to discover).
1. Try clustering categorical variables by their relation to Radon levels
1. Try replacing categorical values with their level frequencies or other encodings
1. Try scaling features linearly or nonlinearly
1. Try embedding **textual** values (eg. US States names) with pre-trained SBERT-like models. This injects some additional information from Wikipedia (or whichever corpora were used for model training).
1. Do EDA and understand the variables and their relation to the output. [Example 1](https://docs.pymc.io/en/v3.11.4/pymc-examples/examples/case_studies/multilevel_modeling.html), [Example 2](https://www.tensorflow.org/probability/examples/Multilevel_Modeling_Primer)

<hr>
<font color=black>
    <details><summary><font color=carnelian>▶ </font>Clustering categorical variables <b></b>.</summary>

  1. When we represent categorical variables as dummies, we may be losing important multivariate information. For example, say we use weekdays to predict the number of hours a person works. We could convert weekdays to 6 features (one is dropped due to collinearity). This requires 6 coefficients (degrees of freedom or sources of uncertainty). Essentially, we have an overparameterized model, whereas all we really need is two clusters of categorical values - weekends (Sat/Sun) and non-weekends (M/T/W/Th/F). In general, the model overparameterized model will do worse due to higher variance of the model output (resulting from the overfit and higher flexibility).

  1. Here is another example from the NLP domain, where each word is a feature (or dimension). While morphological variants of a word (eg. run, running, runner, ran, runs, ...) have lower frequency, we cluster them into the same lemma "run", assuming only a small loss of semantic information. We hope that the gain in building a better distribution estimate for "run" is greater than the loss of semantic and lexical information.
        </details>
    <details><summary><font color=carnelian>▶ </font>Distance to Radon source<b></b>.</summary>

If you can determine where Radon is most active (i.e. the source), then you might be able to compute the distance to the source. Ordinarily, we expect lower radiation for greater distance from the source (assuming uniform distribution of underground rivers, geology, rains/winds and other weather conditions affecting distribution of radon, etc.). You could also use categorical features in (e.g. US State, region, etc.), but these might perform better when clustered (again). Distance to the source is a real-valued feature, which does not require clustering.
        </details>
</font>